# 데이터 초기 설정

In [1]:
import pandas as pd
products=pd.read_csv('amazon_baby_subset.csv')

In [2]:
#데이터 없는 부분(na)을 공백으로 채우기
products = products.fillna({'review':''}) 

In [3]:
#특수기호 제거하기(,)
import string
table = str.maketrans(dict.fromkeys(string.punctuation))

def remove_punctuation(text):
      return text.translate(table)
    
products['review_clean']=products['review'].apply(remove_punctuation)

In [4]:
important_words=pd.read_json('important_words.json')

# vector matrix 만들기

In [5]:
#products 데이타프레임에 feature 열을 생성, 각 행(데이타 포인트별)에 해당 feature가 
#언급된 횟수 측정
for word in important_words[0]:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [6]:
def get_numpy_data(dataframe, features,label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = dataframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [7]:
words=list(important_words[0])

In [8]:
feature_matrix, sentiment=get_numpy_data(products,words,'sentiment')

C:\Users\Jung\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Jung\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [10]:
products['sentiment'].value_counts()

-1    115
 1     79
Name: sentiment, dtype: int64

In [28]:
score=np.dot(feature_matrix,np.zeros(194))
score

array([0., 0., 0., ..., 0., 0., 0.])

# probability prediction (모델링 과정 생략)

In [2]:
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients) # N * 1
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1.0/(1+np.exp(-score))
    
    return score

In [3]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    # scores.shape (53072L, 1L)
    # indicator.shape (53072L,)
    lp = np.sum((np.transpose(np.array([indicator]))-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

# prob를 이용하여 derivative 구하기

In [4]:
#하나의 coefficient에 대한 derivative
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    products = [] #데이터 담을 가방
    for num1, num2 in zip(errors, feature):
        products.append(num1 * num2)
        
    derivative = sum(products)
        # Return the derivative
    return derivative

In [5]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(np.transpose(errors), feature)
    # Return the derivative
    derivative=sum(derivative)
    return derivative

# 이전 것들 귀찮으면 이걸로

In [1]:
import numpy as np
import pandas as pd

feature_csv=pd.read_csv('feature_matrix.csv')
feature_csv=feature_csv.drop(['Unnamed: 0'],axis=1)
feature_matrix=np.array(feature_csv)

sent=pd.read_csv('sentiment.csv')
sent=sent.drop(['Unnamed: 0'],axis=1)
sentiment=np.array(sent)

# Gradient Descent 이용한 Logistic Regression : 계수 예측

In [6]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors a8s indicator - predictions
        errors = np.transpose(np.array([indicator])) - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative

            derivative = feature_derivative(errors , feature_matrix[:,j])
            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j]+=(step_size*derivative)
        # Checking whether log likelihood is increasing

    return coefficients

In [7]:
import numpy as np
from math import exp
coefficients=logistic_regression(feature_matrix, sentiment, np.zeros(194), 1e-7, 301)

MemoryError: 

In [12]:
product['pred']=predictions
product.loc[product['pred']==product['sentiment']]

NameError: name 'product' is not defined

NameError: name 'xrange' is not defined

In [49]:
# coefficient 수치와 word_vector 연결하기
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(list(important_words[0]), coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)